In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from dataclasses import dataclass
import pandas_datareader as pdr
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime


In [2]:
#processing data before passing it to the function
df = pd.read_csv("E:\products\Trading_bots\GBPNZD60.csv")
df.columns=['date', 'time', 'open', 'high', 'low', 'close', 'volume']
df = pd.DataFrame(df)
df['dateTime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
df = df[['dateTime','open', 'high', 'low', 'close', 'volume']]

# Getting data from csv or yfinance
def get_data(data: str=None, tick=None, start: str=None, end: str=None, period: str=None ):
   if data is not None:
      #df = pd.read_csv(data)
      df = data
      #df.columns=['DateTime','Open', 'High', 'Low', 'Close', 'volume']
      #Check if NA values are in data
      df=df[df['volume']!=0]
      df.reset_index(drop=True, inplace=True)
      df.isna().sum()
      df.head(10)
   else:
      # download dataframe
      df = pdr.get_data_yahoo(tick, period=period, start=start, end=end)
      df.head(10)
      #GBPNZD=X
   return df

df = get_data(df)

      

In [3]:
#calculate the point
def pivotid(df1, l, n1, n2): #n1 n2 before and after candle l
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0
    
    pividlow=1
    pividhigh=1
    for i in range(l-n1, l+n2+1):
        if(df1.low[l]>df1.low[i]):
            pividlow=0
        if(df1.high[l]<df1.high[i]):
            pividhigh=0
    if pividlow and pividhigh:
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0
    
df['pivot'] = df.apply(lambda x: pivotid(df, x.name,10,10), axis=1)

In [4]:
# Set points to the keylevels
import numpy as np
def pointpos(x):
    if x['pivot']==1:
        return x['low']-1e-3
    elif x['pivot']==2:
        return x['high']+1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [5]:
dfpl = df[-300:-1]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'],
                increasing_line_color= 'green', 
                decreasing_line_color= 'red')])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")
fig.update_layout(xaxis_rangeslider_visible=False)
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.update_layout(paper_bgcolor='black', plot_bgcolor='black')

fig.show()